In [1]:
import os 


In [2]:
%pwd

'd:\\ML projects\\Chicken-Disease-Classification-\\research'

In [3]:
#change the directory to the root folder 
os.chdir("../")

In [4]:
%pwd

'd:\\ML projects\\Chicken-Disease-Classification-'

In [5]:
#entity folder conatins code for what should be the return type of the function 

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path 

In [11]:
%pwd

'd:\\ML projects\\Chicken-Disease-Classification-'

In [25]:
from src.cnnClassifier.constants import *

In [31]:
!pip install -e .


Obtaining file:///D:/ML%20projects/Chicken-Disease-Classification-
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: cnnClassifier
    Found existing installation: cnnClassifier 0.0.0
    Uninstalling cnnClassifier-0.0.0:
      Successfully uninstalled cnnClassifier-0.0.0
  Running setup.py develop for cnnClassifier
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Legacy editable install of cnnClassifier==0.0.0 from file:///D:/ML%20projects/Chicken-Disease-Classification- (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457


In [32]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


ModuleNotFoundError: No module named 'cnnClassifier'